In [11]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import rv_continuous, norm, truncnorm, arcsine, chi2, gamma, triang, entropy
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from time import time
from random import shuffle
from copy import deepcopy
import timeit

In [71]:
gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
gift_numbers = np.array([1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200], dtype=int)
gift_ids = [[item +'_'+str(i) for i in range(gift_numbers[index])] for index, item in enumerate(gift_types)]
df = pd.read_csv('best_combinations.csv')

In [58]:
def unshared_copy(inList):
    if isinstance(inList, list):
        return list( map(unshared_copy, inList) )
    return inList

def choiceful_gnome(dataframe, gift_numbers, N_packs=1000):
    dataframe = dataframe.copy().reset_index(drop=True)
    packs_left = N_packs
    items_left = gift_numbers[:]
    cart_recipes = []
    cart_scores = []
    while packs_left != 0:
        max_packs = dataframe.apply(lambda x: min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        max_index = total_score.idxmax()
        n = int(max_packs[max_index])
        if n ==0:
            print('!!!Incomplete cart!!!')
            print('Cart length = %.2d\nCart average = %.2f' % (len(cart_recipes),sum(cart_scores)/len(cart_recipes)))
            break
        max_score = scores[max_index]
        max_recipe = dataframe.iloc[max_index,:9]
        cart_recipes += [list(map(int,max_recipe))] * n
        cart_scores += [max_score] * n
        packs_left -= n
        items_left -= max_recipe * n
        if min(items_left)<0:
            print(packs_left)
            print(max_packs[max_index])
            #print(cart_recipes)
            print('packs_left = %d\n' % packs_left)
            print('n = %d' % n)
            print('max_score = %d' % max_score)
            print('max_index = %d' % max_index)
            print('max_recipe = %s\n' % list(max_recipe))
            print('items_left =\n%s' % list(items_left))
            raise Exception('Something went wrong')
    return [cart_scores, cart_recipes, list(items_left.astype(int))]
        
        
        
        

In [178]:
from collections import OrderedDict

class Recipe:
    def __init__(self, recipe=None, score=None, row=None):
        if row is not None:
            self.setrow(row)
        else:
            self.recipe = self.setr(recipe)
            self.score = score
    
    def __repr__(self):
        return 'Recipe = ' + str(self.recipe) + ', Score = ' + str(self.score)

    def __gt__(self, other):
        return self.score() > other.score()
    
    def __lt__(self, other):
        return self.score() < other.score()
        
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        return NotImplemented

    def __ne__(self, other):
        """Define a non-equality test"""
        if isinstance(other, self.__class__):
            return not self.__eq__(other)
        return NotImplemented

    def __hash__(self):
        """Override the default hash behavior (that returns the id or the object)"""
        return hash(tuple(sorted(self.__dict__.items())))
    
    def getr(self):
        return self.recipe
    
    def gets(self):
        return self.score
    
    def setr(self, recipe):
        self.recipe = tuple([int(i) for i in recipe])
    
    def setrow(self,row):
        self.recipe = tuple(row[:9])
        self.score = row[9]
    
    
class Cart:
    cart_df = df.copy()
    cart_df = cart_df[cart_df['n_items']>=3].reset_index(drop=True)

    def __init__(self, recipe_list=None, recipe_number=None, cart=None, gift_numbers=None,items_left=None, gift_types=None, max_size=1000):
        if cart is not None:
            cart = deepcopy(cart)
            self.items_left = cart.items_left
            self.gift_numbers = cart.gift_numbers
            self.gift_types = cart.gift_types
            self.max_size = cart.max_size
            self.cart = cart.cart
            return
        if recipe_list is None:
            recipe_list = []
        if recipe_number is None:
            recipe_number = [1] * len(recipe_list)
        if gift_numbers is None:
            gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
        if items_left is None:
            items_left = list(gift_numbers)
        if gift_types is None:
            gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
        if len(items_left) != len(gift_types):
            raise Excpetion('items_left and gift_types have different lengths.')
            
        self.items_left = list(items_left)
        self.gift_numbers = list(gift_numbers)
        self.gift_types = list(gift_types)
        self.max_size = max_size
        self.cart = dict()
        
        for r,n in zip(recipe_list, recipe_number):
            self.add(r,n)
    
    def __len__(self):
        return sum(self.cart.values())
    
    def __repr__(self):
        out_str = ''
        for r in self.cart:
            out_str += str(r) + ', Instances = ' + str(self.cart[r]) + '\n'
        out_str += 'Items left = ' + str(self.items_left) + '\n'
        out_str += 'Cart size = ' + str(len(self)) + '\n'
        out_str += 'Cart score = ' + str(self.getscore())
        return out_str
    
    def __gt__(self, other):
        return self.getscore() > other.getscore()
    
    def __lt__(self, other):
        return self.getscore() < other.getscore()
    
    def __eq__(self, other):
        return self.getscore() == other.getscore()
    
    def getscore(self):
        score = 0
        for r,n in self.cart.items():
            score += r.gets() * n
        return score
                   
    def set_items_left(self, items_left):
        if len(items_left) != 9:
            raise Exception('Length of items_left must be 9.')
        self.items_left = items_left
        
    def getrecipes(self):
        return [a for a,b in self.cart.items()]
    
    def add(self, recipe, n=1, addmax=False):
        if addmax:
            n = int(self.maxr(recipe))
        else:
            n = int(n)
        self.items_left = [i-r*n if (i-r*n)>=0 else exec('raise Exception(\'Negative number of items left.\')') for i,r in zip(self.items_left, recipe.getr())]
        if recipe in self.cart:
            self.cart[recipe] += n
        else:
            self.cart[recipe] = n
            
    def add_max(self, recipe, n=1):
        n = int(min(n, self.maxr(recipe)))
        self.items_left = [i-r*n if (i-r*n)>=0 else exec('raise Exception(\'Negative number of items left.\')') for i,r in zip(self.items_left, recipe.getr())]
        if recipe in self.cart:
            self.cart[recipe] += n
        else:
            self.cart[recipe] = n
    
    def remove(self, recipe, n=1, removeall=False):
        if recipe not in self.cart:
            raise Exception('Item not in cart.')
        if removeall:
            n = self.cart[recipe]
        else:
            n = int(n)
        if self.cart[recipe] < n:
            raise Exception('Tried to remove too many items.')
        elif self.cart[recipe] == n:
            del self.cart[recipe]
        else:
            self.cart[recipe] -= 1
        self.items_left = [i+r*n for i,r in zip(self.items_left, recipe.getr())]
        
    def maxr(self, recipe):
        packs_left = self.max_size - len(self)
        return min([packs_left] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(self.items_left, recipe.getr())])

    def max_packs(self):
        packs_left = self.max_size - len(self)
        return np.minimum(np.amin(np.divide(self.items_left, self.cart_df.iloc[:,:9]), axis=1).astype(int),packs_left)
    
    @staticmethod
    def _multiple_pop(li, npop):
        out = []
        npop = int(npop)
        for i in range(npop):
            out += [li.pop()]
        return out    
    
    def _recipe_list_to_ids(self, recipe, gift_ids):
            return ' '.join([' '.join(self._multiple_pop(gift_ids[i], item_number)) for i,item_number in enumerate(recipe.getr()) if item_number > 0])
  
    def get_gift_list(self, random=False):
        ids = [[item +'_'+str(i) for i in range(self.gift_numbers[index])] for index, item in enumerate(self.gift_types)]
        if random: 
            for i in ids:
                shuffle(i)
        out = []
        for r,n in self.cart.items():
            out += [self._recipe_list_to_ids(r, ids) for i in range(n)]
        return out
    
    def best_recipes(self, depth):
        dataframe = self.cart_df
        max_packs = dataframe.apply(lambda x: min([self.max_size - len(self)] + [(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(self.items_left, x[:9])]),axis=1)
        scores = dataframe['score']
        total_score = max_packs * scores
        best_scores_indices = list(total_score.sort_values(ascending=False).index[0:depth])
        out = []
        for i in best_scores_indices:
            new_recipe = Recipe(row=dataframe.iloc[i])
            out.append(new_recipe)
        return out
    
    def best_recipes_max_n(self, depth, max_n):
        dataframe = self.cart_df
        max_packs = np.minimum(self.max_packs(), max_n)
        scores = dataframe['score']
        total_score = max_packs * scores
        best_scores_indices = list(total_score.sort_values(ascending=False).index[0:depth])
        out = []
        for i in best_scores_indices:
            new_recipe = Recipe(row=dataframe.iloc[i])            
            if max_packs.iloc[i] == 0:
                return out
            out.append(new_recipe)
        return out
    
    
    def isfull(self):
        if len(self) == self.max_size:
            return True
        elif len(self) > self.max_size:
            raise Exception('Max size exceeded.')
        else:
            return False

In [60]:
def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

def test(depth):
    if depth == 0:
        return [depth]
    else:
        return flatten(list(map(lambda x: test(x-1), [depth]*3)))

In [209]:
def mask(df, f):
    """df.mask(lambda x: x[0] < 0).mask(lambda x: x[1] > 0)"""
    return df[f(df)]

def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

#number_of_iterations = 1
#finished_leaves = 1
#finished_carts = []

def searchful_gnome(cart, depth, max_n):
    global number_of_iterations
    global finished_leaves
    global finished_carts

    
    #####
    print('Iteration number: %d, cart length = %d' % (number_of_iterations, len(cart)))
    number_of_iterations+=1
    #####
    
    if cart.isfull():
        #####
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        finished_carts.append(cart)
        #####
        return [cart]  
    
    best_recipes = cart.best_recipes_max_n(depth, max_n)
    
    
    if best_recipes == []:
        print('!!!Cart is not full!!!')
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        finished_carts.append(cart)
        return [cart]
    
    new_carts = []
    for br in best_recipes:
        nc = deepcopy(cart)
        nc.add_max(br, max_n)
        new_carts.append(nc)
    
    out = []
    return flatten(list(map(searchful_gnome, new_carts, [depth]*depth, [max_n]*depth)))
    
lazy_finished_carts = []

def get_depth(dlist, n):
    l = len(dlist)-1
    if n>l:
        n=l
    return dlist[n]

def lazy_searchful_gnome(cart, depth, max_n):
    global number_of_iterations
    global finished_leaves
    global lazy_finished_carts
    
    
    #####
    print('Iteration number: %d, cart length = %d, depth = %d' % (number_of_iterations, len(cart), depth))
    number_of_iterations+=1
    #####
    
    if cart.isfull():
        #####
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        lazy_finished_carts.append(cart)
        #####
        return [cart]  
    
    if (len(cart)>=950):
        if((cart.getscore()/len(cart))<35.6):
            print('Number of finished leaves: %d' % finished_leaves)
            print('!!!Aborted cart!!!')
            finished_leaves+=1
            lazy_finished_carts.append(cart)
            return [cart]
    
    best_recipes = cart.best_recipes_max_n(depth, max_n)
    
    
    if best_recipes == []:
        print('Number of finished leaves: %d' % finished_leaves)
        print('!!!Cart is not full!!!')
        finished_leaves+=1
        lazy_finished_carts.append(cart)
        return [cart]
    
    new_carts = []
    for br in best_recipes:
        nc = deepcopy(cart)
        nc.add_max(br, max_n)
        new_carts.append(nc)
    
    out = []
    if depth >1: depth -= 1
    for c in new_carts:
        out.append(lazy_searchful_gnome(c, depth, max_n))
    return out
    #return flatten(list(map(lazy_searchful_gnome, new_carts, [depth]*depth, [max_n]*depth)))
    
        


def lazy_searchful_gnome_depth(cart, depth_list, max_n, depth_index=0):
    global number_of_iterations
    global finished_leaves
    global lazy_finished_carts
    
    depth = get_depth(depth_list,depth_index)
    
    #####
    print('Iteration number: %d, cart length = %d, depth = %d' % (number_of_iterations, len(cart), depth))
    number_of_iterations+=1
    #####
    
    if cart.isfull():
        #####
        print('Number of finished leaves: %d' % finished_leaves)
        finished_leaves+=1
        lazy_finished_carts.append(cart)
        #####
        return [cart]  
    
    if (len(cart)>=950):
        if((cart.getscore()/len(cart))<35.6):
            print('Number of finished leaves: %d' % finished_leaves)
            print('!!!Aborted cart!!!')
            finished_leaves+=1
            lazy_finished_carts.append(cart)
            return [cart]
    
    best_recipes = cart.best_recipes_max_n(depth, max_n)
    
    
    if best_recipes == []:
        print('Number of finished leaves: %d' % finished_leaves)
        print('!!!Cart is not full!!!')
        finished_leaves+=1
        lazy_finished_carts.append(cart)
        return [cart]
    
    new_carts = []
    for br in best_recipes:
        nc = deepcopy(cart)
        nc.add_max(br, max_n)
        new_carts.append(nc)
    

    depth_index+=1
    return flatten(list(map(lazy_searchful_gnome_depth, new_carts, [depth_list]*depth, [max_n]*depth, [depth_index]*depth)))

In [163]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
#with open('best_cart.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
#    pickle.dump(best_cart, f)

# Getting back the objects:
with open('best_cart.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    best_cart = pickle.load(f)[0]

In [9]:
best_cart

[Recipe = (0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 1.0), Score = 36.4783336725, Instances = 80
 Recipe = (1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 39.1145416738, Instances = 80
 Recipe = (0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0), Score = 33.7878330755, Instances = 100
 Recipe = (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0), Score = 37.8699968782, Instances = 80
 Recipe = (3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.1926429875, Instances = 80
 Recipe = (2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.7720062927, Instances = 80
 Recipe = (0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0), Score = 30.424757689, Instances = 100
 Recipe = (0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0), Score = 33.1341166931, Instances = 200
 Recipe = (2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0), Score = 38.0301439692, Instances = 100
 Recipe = (0.0, 1.0, 0.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0), Score = 35.4460171264, Instances = 20
 Recipe = (3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0

In [ ]:
def quick_max_packs(cart):
    

def cart_neighbors(cart):
    N_max = 1000
    if len(cart) == N_max:
        return []
    

In [10]:
df

,horse,ball,bike,train,coal,book,doll,blocks,gloves,score,n_items,max_packs
0,0,23,0,0,0,0,0,0,0,45.863013,23,47
1,0,22,0,0,0,0,0,0,1,44.875672,23,50
2,1,20,0,0,0,0,0,0,0,44.112728,21,55
3,0,21,0,0,0,0,0,0,2,44.002327,23,52
4,0,22,0,0,0,0,0,0,0,43.999495,22,50
5,0,21,0,0,0,1,0,0,0,43.659044,22,52
6,0,20,0,0,0,0,1,0,0,43.592813,21,55
7,0,21,0,0,0,0,0,0,1,43.380163,22,52
8,1,19,0,0,0,0,0,0,1,43.357960,21,57
9,0,20,0,0,0,0,0,0,3,43.260245,23,55


In [54]:
%%timeit
for i in df.iterrows():
    i[:9]

1 loop, best of 3: 522 ms per loop


In [56]:
%%timeit
for i in df.itertuples():
    min(i[:9])

10 loops, best of 3: 42.8 ms per loop


In [117]:
cart = deepcopy(best_cart)

In [90]:
%%timeit
np.amin(np.divide(gift_numbers, df.iloc[:,:9]), axis=1).astype(int)

100 loops, best of 3: 6.16 ms per loop


In [101]:
%%timeit
df.apply(lambda x: min([(n_left//n_recipe) if n_recipe>0 else np.inf for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)

1 loop, best of 3: 2.62 s per loop


In [105]:
%%timeit
np.minimum(np.amin(np.divide(gift_numbers, df.iloc[:,:9]), axis=1).astype(int),5)

100 loops, best of 3: 6.11 ms per loop


In [111]:
a = Cart()

In [113]:
%%timeit
a.max_packs()

100 loops, best of 3: 6.19 ms per loop


In [180]:
c = Cart(cart=best_cart)

In [184]:
rec = c.getrecipes()
rec

[Recipe = (1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 39.1145416738,
 Recipe = (0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0), Score = 33.7878330755,
 Recipe = (0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 1.0), Score = 36.4783336725,
 Recipe = (3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.1926429875,
 Recipe = (0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0), Score = 30.424757689,
 Recipe = (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0), Score = 37.8699968782,
 Recipe = (0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0), Score = 33.1341166931,
 Recipe = (2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0), Score = 38.0301439692,
 Recipe = (0.0, 1.0, 0.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0), Score = 35.4460171264,
 Recipe = (2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.7720062927,
 Recipe = (3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0), Score = 37.4444221882]

In [185]:
c

Recipe = (1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 39.1145416738, Instances = 80
Recipe = (0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0), Score = 33.7878330755, Instances = 100
Recipe = (0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 1.0), Score = 36.4783336725, Instances = 80
Recipe = (3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.1926429875, Instances = 80
Recipe = (0.0, 0.0, 1.0, 1.0, 0.0, 2.0, 0.0, 0.0, 1.0), Score = 30.424757689, Instances = 100
Recipe = (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0), Score = 37.8699968782, Instances = 80
Recipe = (0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0), Score = 33.1341166931, Instances = 200
Recipe = (2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0), Score = 38.0301439692, Instances = 100
Recipe = (0.0, 1.0, 0.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0), Score = 35.4460171264, Instances = 20
Recipe = (2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.7720062927, Instances = 80
Recipe = (3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0), S

In [200]:
for i in range(20):
    c.remove(rec[10])
c

Recipe = (1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 39.1145416738, Instances = 40
Recipe = (0.0, 1.0, 0.0, 3.0, 0.0, 0.0, 1.0, 0.0, 0.0), Score = 33.7878330755, Instances = 60
Recipe = (0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 4.0, 1.0, 1.0), Score = 36.4783336725, Instances = 60
Recipe = (3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.1926429875, Instances = 60
Recipe = (1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0), Score = 37.8699968782, Instances = 60
Recipe = (2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0), Score = 38.0301439692, Instances = 80
Recipe = (0.0, 1.0, 0.0, 0.0, 0.0, 8.0, 4.0, 0.0, 0.0), Score = 35.4460171264, Instances = 20
Recipe = (2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.7720062927, Instances = 60
Recipe = (3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0), Score = 37.4444221882, Instances = 60
Items left = [260.0, 400.0, 500.0, 820.0, 166.0, 860.0, 240.0, 280.0, 140.0]
Cart size = 500
Cart score = 18668.6276327

In [222]:
number_of_iterations = 1
finished_leaves = 1
finished_carts = []
d = searchful_gnome(Cart(), 8, 300)

Iteration number: 1, cart length = 0
Iteration number: 2, cart length = 300
Iteration number: 3, cart length = 600
Iteration number: 4, cart length = 900
Iteration number: 5, cart length = 1000
Number of finished leaves: 1
Iteration number: 6, cart length = 1000
Number of finished leaves: 2
Iteration number: 7, cart length = 1000
Number of finished leaves: 3
Iteration number: 8, cart length = 1000
Number of finished leaves: 4
Iteration number: 9, cart length = 1000
Number of finished leaves: 5
Iteration number: 10, cart length = 1000
Number of finished leaves: 6
Iteration number: 11, cart length = 1000
Number of finished leaves: 7
Iteration number: 12, cart length = 1000
Number of finished leaves: 8
Iteration number: 13, cart length = 850
Iteration number: 14, cart length = 1000
Number of finished leaves: 9
Iteration number: 15, cart length = 1000
Number of finished leaves: 10
Iteration number: 16, cart length = 950
Iteration number: 17, cart length = 1000
Number of finished leaves: 11

RecursionError: maximum recursion depth exceeded in comparison

In [220]:
rd = list(reversed(sorted(finished_carts)))

In [221]:
rd[0]

Recipe = (0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0), Score = 31.7450125139, Instances = 112
Recipe = (0.0, 2.0, 1.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0), Score = 27.769552453, Instances = 46
Recipe = (0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0), Score = 26.3030204831, Instances = 11
Recipe = (3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0), Score = 38.1926429875, Instances = 1
Recipe = (0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, 0.0, 0.0), Score = 33.1341166931, Instances = 166
Recipe = (0.0, 0.0, 0.0, 1.0, 0.0, 3.0, 4.0, 0.0, 1.0), Score = 35.0197146126, Instances = 166
Recipe = (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 3.0, 0.0), Score = 37.7179237292, Instances = 332
Recipe = (6.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), Score = 39.4023671413, Instances = 166
Items left = [1.0, 0.0, 443.0, 0.0, 166.0, 0.0, 4.0, 2.0, 1.0]
Cart size = 1000
Cart score = 35537.046303